### Shadman Ahmed, Oguzhan Ugur

## Notice
You run the code by running all the cells, the last three cells prints the results

## Finding Similar Items: Textually Similar Documents  

### Problem Formulation
 
In this assignment, we were instructed to implement an application that finds similarities in documents based on Jacard similarity using shingling, minhashing, and locality-sensitive hashing. The lab descriptions instructs us to create five classes in order to construct this similarity detector:
- Class 1: Shingling
- Class 2: JacardSimilarity
- Class 3: Minhashing
- Class 4: CompareSignatures
- Class 5: LSH

### Data
The data used in this assignment were 10 different text documents. These documents are stored in the directory named "Data" and was taken from UCI- opinosis Opinion/Review. 

### Shingling
K-shingling is a method that divides the document into several parts, in other words it converts documents into sets(shingles). We created a Shingle class that has 3 functions:<br>
- First _FileRead_, that reads the file, filters the punctuations and returns the text without punctuations.<br>
- Second _CreateShingles_, that creates the shingles and returns a list with all of the shingles for respective document. Consider that it is possible to choose the shingle size by changing __k__.<br>
- Third _CreateHash_, that hashes the shingles that was stored in a list and returns a list with hash values corresponding to respective shingles and the name of the document. The hashlib library (md5) was used in order to create the hash values. The name of the document is returned because we wanted to store all the hashed values in a dictionary with the name of the document as key value. 

    

In [3]:
import hashlib
import os
import string 
import numpy as np
import pandas as pd
import time

In [4]:

class Shinling:
    #k- number of shingle
    #doc- name of the document
    def __init__(self, doc, k):
        self.k = k
        self.doc = doc
        
    #Reads the file
    def FileRead(self):
        
        with open(self.doc, "r") as file: 
            text = file.read()
            cleanText = ""
            for character in text:
                if character not in string.punctuation:
                    cleanText = cleanText + character
        return (cleanText)
    
    #Creates shingles of the cleaned text(text without punctuation)
    def CreateShingles(self):
        text = self.FileRead()
        
        shingle_list = []
        for index in range(len(text) - self.k + 1):
            shingle = text[index:index + self.k]
            shingle_list.append(shingle)
        shingle_list = [x.replace('\n', '') for x in shingle_list]
        shingle_list = set(shingle_list)
        
        return(shingle_list)
    
    #Creates Hash values for each shingle
    def CreateHash(self):
        shingle_list = self.CreateShingles()
        
        shingles_in_document = set()
        for shingles in shingle_list:
            hashvalue = hashlib.md5(shingles.encode())
            shingles_in_document.add(hashvalue.hexdigest())   
        return(shingles_in_document,self.doc)
        
       
        

### Jaccard Similarity
Jaccard similarity is a distance measure that can be used to show the similarity and diversity of two sets. In our case this method was used in order to determine the similarity between the shingle sets for each document. The Jaccard similarity score is greater for small shingle sizes, on the other hand, large shingle sizes corresponds to smaller jaccard similarity scores, because the documents we use are different. The Jaccard similarity is computed by taking the intersection between two sets and dividing it by the union of these sets. <br>
<br>
The Jaccard_Similarity Class consist of one function, "jaccard\_similarity" , that returns the similarity score after calculating the math of the two shingle sets. This function takes _"document"_ and _"name_doc"_ as input arguments. _document_ is a dictionary that consists of hashed values and corresponding names of each document with the document name as key. The variable name_doc is a list with the names of the documents. 


In [5]:
class Jacard_Similarity:
    # documant - is a dictionary, with keys as document names 
    #            and values as hash values of the shingles
    # name_doc - is the name of the documents
    def jacard_similarity(self,document,name_doc):

        similarity_score = {}
        for i in range(len(document)):
            shingle_set1= document[name_doc[i]]
            for j in range(i+1,len(document)): 
                shingle_set2 = document[name_doc[j]]
                intersection = len(shingle_set1.intersection(shingle_set2))
                union = (len(shingle_set1)+len(shingle_set2))-intersection
                score = intersection/union
                name = name_doc[i]+ "  vs  " + name_doc[j]
                similarity_score[name] = score
                
        return(similarity_score)
 
 

       
        

### Minhashing

MinHasing is a method that quickly estimates how similar two sets are. The main idea is to use a charesteristic matrix that consists of zeros and ones. In our case, the rows of the charesteristic matrix  represents shingles and the columns represents the documents. If a shingle appears to be in a document, then the column representing the document will have 1 on the row that represents the shingle and 0 the other way around. This matrix will then be compressed to a signature matrix by using random permuation with hash functions. This signature matrix will then be used in order to determine the similarity between the documents(columns). The number of rows of the signature matrix, will correspond to the number of hash functions. The signature matrix consists of the smallest hashvalue received by the hashfunction for each index of the charesteristic matrix. 

The minHash Class consist of four functions, _"createCharMatrix"_, _"hashFunction"_, _"SignatrueMatrix"_. <br> 
- First, The createCharMatrix_ creates the charesteristic matrix by comparing the shingles in the universal set with the single set for each document. The Universal set, is a set with shingles from all of the documents.<br>
- Second, The _hashFunction_ returns a list of hash functions with random constants, the hashfunction is equal to ax+b % c where a and b are chosen randomly with the condidtion to be less than the variable maxShingle. And c is a primenumber slightly above the maxShingle number. 
- Third, The _SignatureMatrix_ creates the signature matrix by using the hash functions and the charesteristic matrix. 

In [9]:
class minHash:
    def __init__(self,doc_hashed_shingle,UniversalSet,numberOfHashFunction):
        self.doc_hashed_shingle = doc_hashed_shingle
        self.UniversalSet = UniversalSet
        self.numberOfHashFunction = numberOfHashFunction
    
    def createCharMatrix(self):
        charact_Matrix = np.zeros((len(self.UniversalSet),len(self.doc_hashed_shingle)))

        for index_i,i in enumerate(self.UniversalSet):
            for index_j,j in enumerate(self.doc_hashed_shingle):
                if i in self.doc_hashed_shingle[j]:
                     charact_Matrix[index_i,index_j] = 1
                else:
                    charact_Matrix[index_i,index_j] = 0
        return (charact_Matrix)
        
    def hashFunction(self,seed = 31):
        maxShingle = 4294967295
        
        c = 4294967311
        
        np.random.seed(seed)
        a = np.random.randint(0,maxShingle,self.numberOfHashFunction, dtype = "int64")
        b = np.random.randint(0,maxShingle,self.numberOfHashFunction, dtype = "int64")
        
        hashFunctionList = []
        for i in range(self.numberOfHashFunction):
            h = lambda x, j=i: (a[j]*x + b[j])%c
            hashFunctionList.append(h)
        
        return (hashFunctionList)
    
    def SignatureMatrix(self):
        hashFunctions = self.hashFunction()
        CharMatrix = self.createCharMatrix()
        
        signMatrix = np.full((len(hashFunctions),len(self.doc_hashed_shingle)),np.inf)
        
        for i in range(0,len(self.UniversalSet)):
            hash_f = []
            for hashfunction in hashFunctions:
                hash_1 = hashfunction(i)
                hash_f.append(hash_1)
            for j in range(0,len(self.doc_hashed_shingle)):
                if(CharMatrix[i,j]==1):
                    for hf_Index,hashValue in enumerate(hash_f):
                        if(hashValue < signMatrix[hf_Index,j]):
                            signMatrix[hf_Index,j] = hashValue
        return (signMatrix)
        
        
        

### Similarity of Signatures

The similarity of signatures is the fraction of the minhash function(rows) in which they agree. The expected similarity of two signatures equals the Jaccard similarity of the columns or the sets that the signatures represent.

The class CompareSignatures consist of one function, "similarity\_of\_sing". This function returns the similarity score of the signatures. The similarity score between the signatures was determined by comparing 2 columns at a time. For each similarity, +1 was added to a variable named _score_, the total score was then divided by the total number of rows.  


In [10]:
class CompareSignatures:
    
    def similarity_of_sign(self, signMatrix, nameList):
        
        similarity_score = {}
        for i in range(signMatrix.shape[1]):
            c1= signMatrix[:,i]
            for k in range(i+1, signMatrix.shape[1]):
                score = 0
                c2= signMatrix[:,k]
                for j in range(signMatrix.shape[0]):
                    if c1[j] == c2[j]:
                        score = score + 1
        
                
                name = nameList[i]+ "  vs  " + nameList[k]
                similarity_score[name] = score/signMatrix.shape[0]
                
        return(similarity_score)

# LSH

Local sensitive hashing is a method that reduces the complexity by introducing candidate pairs. It basically reduces the dimensionality of high-dimensional data. The LSH maps similar items to buckets by hashing these items. This mapping is conducted by using the signature matrix. The number of buckets is equal to the number of Bands. Bands are partitions of the signature matrix. The columns of these bands are then hashed in order to find similarities between the columns of the bands. If there is any similarity then they are stored in a bucket. The similarity score is then computed as we did before but only for the candidate pairs. 

The class LSH consist of four functions,<br>
- First, The _hashToBuckets_ function returns the buckets with the hashed columns. We chose to map all columns of the bands into the buckets, so we didn't store the similar columns at this stage. 
- Second, The _CandidatePairs_ function returns a dictionary with the candidate pairs (have same hash value) for each bucket. So, we chose to filter out the non pairs after we mapped all of the hashed columns into the buckets. 
- Third, The _SimilarityScore_ function returns a dictionary with the scores for the different columns(documents). The key in this dictionary corresponds to the names of the compared documents and the value is the similarity score. 
- Fourth, The _SimilarDocuments_ function introduces a threshold that filters out the scores that are underneath the threshold and returns a dictionary with the scores above it. 



In [38]:
class LSH:
    
    def __init__(self, signMatrix, r):
        self.signMatrix = signMatrix
        self.r = r
        
        
    def hashToBuckets(self):
        hashed_buckets = {}
        sign_matrix = self.signMatrix
        for i in range(int(sign_matrix.shape[0]/self.r)):
            hashed_bucket = {}
            band = sign_matrix [i*self.r:i*self.r+self.r]
            for j in range(band.shape[1]):
                colband = str(band[:,j])
                hashvalue = hashlib.md5(colband.encode())
                hashValue = hashvalue.hexdigest()
                hashed_bucket[j] = hashValue
            hashed_buckets["bucket " + str(i)] = hashed_bucket
        return(hashed_buckets)
    
    def CandidatePairs(self):
        hashed_buckets = self.hashToBuckets()
        pairFound = {}

        for index, key in enumerate(hashed_buckets):
            sort_pairs = {}
            hashed_values = hashed_buckets[key]
            for k,v in hashed_values.items():
                sort_pairs.setdefault(v, []).append(k)
            for i,j in sort_pairs.items():
                if len(j)>1:
                    pairFound["Bucket " + str(index)]= j
                else:
                    pairFound["Bucket " + str(index)]= None           
        return(pairFound) 
  

    def SimilarityScore(self):
        candidatePairs = self.CandidatePairs()
        SimilarityScore = {}
        signature_matrix = self.signMatrix
        for i,j in candidatePairs.items():
            if j != None:
                for l in range(len(j)):
                    c1 = signature_matrix[:,j[l]]
                    for m in range(l+1, len(j)):
                        c2 = signature_matrix[:,j[m]]
                        score = 0
                        for k in range(len(c1)):
                            if c1[k] == c2[k]:
                                score = score + 1
                        SimilarityScore[(j[l],j[m])]= score/len(c1) 
        return(SimilarityScore)
    
    
    def SimilarDocuments(self,namesofDocument, threshold):
        similarity_score = self.SimilarityScore()
        score_of_pairs = {}
        for i,j in similarity_score.items():    
            if j >= threshold:
                score_of_pairs[namesofDocument[i[0]] + " vs " + namesofDocument[i[1]]] = j   
        
        return(score_of_pairs)
       

# Result of Jacard similarity for shingles

You can run the code by pressing run or ctrl+enter. <br> You can change the shingle size by changing the variable __k__.

In [29]:
def main_jacardSimilarity_shingling():
    path_to_data = "Data/"
    shingle_doc = {}
    doc_names = []
    k = 2
    
    for documents in os.listdir(path_to_data):
        doc_path = path_to_data + str(documents)
        doc_names.append(doc_path)
        doc_shingle = Shinling(doc_path, k)
        d,name = doc_shingle.CreateHash()
        shingle_doc[name] = d
    
    jacard_similarity = Jacard_Similarity()
    Score = jacard_similarity.jacard_similarity(shingle_doc,doc_names)
    
    format = "{:<100}{:<10}"    
    print( format.format("Documents","Score"))
    for Doc,score in Score.items():
        score = ("%.2f" % score)
        print (format.format(Doc,score))
    
main_jacardSimilarity_shingling()

Documents                                                                                           Score     
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/bathroom_bestwestern_hotel_sfo.txt.data       0.62      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/battery-life_ipod_nano_8gb.txt.data           0.61      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/battery-life_netbook_1005ha.txt.data          0.52      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/comfort_honda_accord_2008.txt.data            0.59      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/comfort_toyota_camry_2007.txt.data            0.61      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/directions_garmin_nuvi_255W_gps.txt.data      0.71      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/display_garmin_nuvi_255W_gps.txt.data         0.65      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/eyesight-issues_amazon_kindle.txt.data        0.65      
D

# Result of signature similarities

You can run the code by pressing run or ctrl+enter
 <br> You can change the shingle size by changing the variable __k__, and the number of hash fucntions by changing nrHashFunc.

In [30]:
def main_minHashing_similarity():
    path_to_data = "Data/"
    shingle_doc = {}
    doc_names = []
    k = 2
    nrHashFunc = 100
    
    for documents in os.listdir(path_to_data):
        doc_path = path_to_data + str(documents)
        doc_names.append(doc_path)
        doc_shingle = Shinling(doc_path, k)
        d,name = doc_shingle.CreateHash()
        shingle_doc[name] = d
    
    UniversalSet = set()
    for i in shingle_doc:
        UniversalSet = UniversalSet.union(shingle_doc[i])
    
    minHashing = minHash(shingle_doc,UniversalSet,nrHashFunc)
    CharM = minHashing.createCharMatrix()
    signature_matrix = minHashing.SignatureMatrix()
    
    compare_signatures = CompareSignatures()
    similarityScore_signature = compare_signatures.similarity_of_sign(signature_matrix,doc_names) 
    
    format = "{:<100}{:<10}"    
    print( format.format("Documents","Score"))
    for Doc,score in similarityScore_signature.items():
        score = ("%.2f" % score)
        print (format.format(Doc,score))
main_minHashing_similarity()


Documents                                                                                           Score     
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/bathroom_bestwestern_hotel_sfo.txt.data       0.62      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/battery-life_ipod_nano_8gb.txt.data           0.61      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/battery-life_netbook_1005ha.txt.data          0.53      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/comfort_honda_accord_2008.txt.data            0.62      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/comfort_toyota_camry_2007.txt.data            0.58      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/directions_garmin_nuvi_255W_gps.txt.data      0.70      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/display_garmin_nuvi_255W_gps.txt.data         0.60      
Data/accuracy_garmin_nuvi_255W_gps.txt.data  vs  Data/eyesight-issues_amazon_kindle.txt.data        0.69      
D

# Result of LSH similarity 

You can run the code by pressing run or ctrl+enter
 <br> You can change the shingle size by changing the variable __k__, and the number of hash fucntions by changing nrHashFunc.<br> 
Band size can be adjusted by changing r, consider that the band size is equal to $\frac{nrHashFunc}{r}$<br>
The threshold of the similarity score for the candidate pairs can be adjusted by changing the threshold variable.

In [39]:
def main_LSH_similarity(n_documents):
    path_to_data = "Data/"
    
    k = 2
    nrHashFunc = 100
    r = 4
    threshold = 0.4
    
    shingle_doc = {}
    doc_names = []
    
    for index, documents in enumerate(os.listdir(path_to_data)):
        if index < n_documents and len(os.listdir(path_to_data)) >= n_documents:
            
            doc_path = path_to_data + str(documents)
            doc_names.append(doc_path)
            doc_shingle = Shinling(doc_path, k)
            d,name = doc_shingle.CreateHash()
            shingle_doc[name] = d
        else:
            break
            
    
    UniversalSet = set()
    for i in shingle_doc:
        UniversalSet = UniversalSet.union(shingle_doc[i])
    
    minHashing = minHash(shingle_doc,UniversalSet,nrHashFunc)
    CharM = minHashing.createCharMatrix()
    signature_matrix = minHashing.SignatureMatrix()
    
    lsh = LSH(signature_matrix, r)
    similarityScore_LSH = lsh.SimilarDocuments(doc_names,threshold)


    format = "{:<100}{:<10}"    
    print( format.format("Documents","Score"))
    for Doc,Score in similarityScore_LSH.items():
        score = ("%.2f" % Score)
        print (format.format(Doc,Score))
        
start_time = time.time()
main_LSH_similarity(2)
print("--- %s seconds ---" %(time.time() - start_time))

{'Bucket 0': None, 'Bucket 1': None, 'Bucket 2': None, 'Bucket 3': None, 'Bucket 4': None, 'Bucket 5': None, 'Bucket 6': None, 'Bucket 7': None, 'Bucket 8': None, 'Bucket 9': None, 'Bucket 10': None, 'Bucket 11': None, 'Bucket 12': None, 'Bucket 13': None, 'Bucket 14': None, 'Bucket 15': None, 'Bucket 16': None, 'Bucket 17': None, 'Bucket 18': [0, 1], 'Bucket 19': None, 'Bucket 20': None, 'Bucket 21': None, 'Bucket 22': None, 'Bucket 23': None, 'Bucket 24': [0, 1]}
Documents                                                                                           Score     
Data/accuracy_garmin_nuvi_255W_gps.txt.data vs Data/bathroom_bestwestern_hotel_sfo.txt.data         0.62      
--- 0.2812483310699463 seconds ---


Finding similar documents in a corpus of 5-10 documents. Testing for time versus input size with a treshold t = 0.7

In [10]:
for i in range(5,11):
    start_time = time.time()
    main_LSH_similarity(i)
    print("Time: %s seconds " %(time.time() - start_time))
    print("Input size: " + str(i) + " documents" )
    print("---------------------------------------")

Documents                                                                                           Score     
Time: 0.703045129776001 seconds 
Input size: 5 documents
---------------------------------------
Documents                                                                                           Score     
Time: 0.7812490463256836 seconds 
Input size: 6 documents
---------------------------------------
Documents                                                                                           Score     
Data/comfort_honda_accord_2008.txt.data vs Data/comfort_toyota_camry_2007.txt.data                  0.72      
Time: 0.8906242847442627 seconds 
Input size: 7 documents
---------------------------------------
Documents                                                                                           Score     
Data/directions_garmin_nuvi_255W_gps.txt.data vs Data/display_garmin_nuvi_255W_gps.txt.data         0.7       
Time: 0.9843931198120117 seconds 
Input s